In [1]:
import pickle
import numpy as np

In [2]:
with open('../results/det_results_coco_oi.pkl', 'rb') as fin:
    det_results_oi = pickle.load(fin)

In [3]:
with open('../results/det_results_coco_1.pkl', 'rb') as fin:
    det_results_coco = pickle.load(fin)

In [18]:
with open('../temp/norm_coef_1.pkl', 'rb') as fin:
    norm_coefs = pickle.load(fin)

In [5]:
with open('../temp/label_matching_coco_oi.pkl', 'rb') as fin:
    label_matching_meta = pickle.load(fin)

In [6]:
idx2mid_oi = label_matching_meta['idx2mid_oi']
idx2mid_coco = label_matching_meta['idx2mid_coco']

In [16]:
def iou(det_bbox, gt_bbox):
    x_d_len = det_bbox[2] - det_bbox[0]
    y_d_len = det_bbox[3] - det_bbox[1]
    x_t_len = gt_bbox[2] - gt_bbox[0]
    y_t_len = gt_bbox[3] - gt_bbox[1]
    x_int_len = max(0, min(gt_bbox[2], det_bbox[2]) - max(gt_bbox[0], det_bbox[0]))
    y_int_len = max(0, min(gt_bbox[3], det_bbox[3]) - max(gt_bbox[1], det_bbox[1]))
    iou = (x_int_len*y_int_len) / (x_d_len*y_d_len + x_t_len*y_t_len - x_int_len*y_int_len)
    return iou

In [31]:
iou_thresh_same = 0.5
iou_thresh_diff = 0.7

det_results_combined = {}
for imgid in det_results_oi:
    merged_coco = set()
    det_results_combined[imgid] = []
    
    for ii in range(det_results_oi[imgid]['num_detections']):
        det_label_oi = idx2mid_oi.get(det_results_oi[imgid]['detection_classes'][ii])
        det_bbox_oi = det_results_oi[imgid]['detection_boxes'][ii]
        det_score_oi = det_results_oi[imgid]['detection_scores'][ii]
        if det_score_oi == 0.:
            continue
        
        det_score_oi_comp_coco = (((det_score_oi - norm_coefs[det_label_oi]['mean_oi']) 
                                   * norm_coefs[det_label_oi]['std_coco'] / norm_coefs[det_label_oi]['std_oi'] 
                                   + norm_coefs[det_label_oi]['mean_coco']) 
                                  if det_label_oi in norm_coefs else (
                                   ((det_score_oi - norm_coefs['default']['mean_oi']) 
                                   * norm_coefs['default']['std_coco'] / norm_coefs['default']['std_oi'] 
                                   + norm_coefs['default']['mean_coco']) 
                                 ))
        
        match = []        
        flag = False
        
        if imgid == '3062896a818e5422':
            print('OI', det_label_oi, det_bbox_oi, det_score_oi)
        
        for iii in range(det_results_coco[imgid]['num_detections']):
            det_label_coco = idx2mid_coco.get(det_results_coco[imgid]['detection_classes'][iii])
            det_bbox_coco = det_results_coco[imgid]['detection_boxes'][iii]
            det_score_coco = det_results_coco[imgid]['detection_scores'][iii]
            if det_score_coco == 0.:
                continue

            if imgid == '3062896a818e5422':
                print('COCO', det_label_oi, det_bbox_oi, det_score_oi)
                
            if det_label_oi == det_label_coco:
                if iou(det_bbox_oi, det_bbox_coco) >= iou_thresh_same:
                    if iii in merged_coco:
                        flag = True
                        break
                    match.append(iii)
                    merged_coco.add(iii)

            else:
                if iou(det_bbox_oi, det_bbox_coco) >= iou_thresh_diff:
                    if det_score_coco > det_score_oi_comp_coco:
                        flag = True
                        break
                    else:
                        merged_coco.add(iii)
                    
        if flag:
            if imgid == '3062896a818e5422':
                print('dumped')
            continue
            
        if len(match) > 0:
            new_bbox = np.asarray([
                min([det_results_coco[imgid]['detection_boxes'][iii][0] for iii in match] + [det_bbox_oi[0]]),
                min([det_results_coco[imgid]['detection_boxes'][iii][1] for iii in match] + [det_bbox_oi[1]]),
                max([det_results_coco[imgid]['detection_boxes'][iii][2] for iii in match] + [det_bbox_oi[2]]),
                max([det_results_coco[imgid]['detection_boxes'][iii][3] for iii in match] + [det_bbox_oi[3]]),
            ])

            new_score = (sum([det_results_coco[imgid]['detection_scores'][iii] for iii in match]) + det_score_oi_comp_coco) / (len(match) + 1)
            new_score = min(max(new_score, 0.), 1.)

            det_results_combined[imgid].append({'detection_class': det_label_oi, 
                                                'detection_box': new_bbox, 
                                                'detection_score': new_score, 
                                                'detection_model': 'joint'})
        else:
            det_results_combined[imgid].append({'detection_class': det_label_oi, 
                                                'detection_box': det_bbox_oi, 
                                                'detection_score': det_score_oi_comp_coco, 
                                                'detection_model': 'oi'})
            
    for iii in set(range(det_results_coco[imgid]['num_detections'])) - merged_coco:
        det_label_coco = idx2mid_coco.get(det_results_coco[imgid]['detection_classes'][iii])
        det_bbox_coco = det_results_coco[imgid]['detection_boxes'][iii]
        det_score_coco = det_results_coco[imgid]['detection_scores'][iii]
        if det_score_coco == 0.:
            continue
        det_results_combined[imgid].append({'detection_class': det_label_coco, 
                                            'detection_box': det_bbox_coco, 
                                            'detection_score': det_score_coco, 
                                            'detection_model': 'coco'})



OI /m/04yx4 [0.45996127 0.29091212 0.9424938  0.5994253 ] 0.45244548
COCO /m/04yx4 [0.45996127 0.29091212 0.9424938  0.5994253 ] 0.45244548
dumped
OI /m/03bt1vf [0.24273162 0.21272242 0.5841096  0.38965252] 0.4436632
COCO /m/03bt1vf [0.24273162 0.21272242 0.5841096  0.38965252] 0.4436632
COCO /m/03bt1vf [0.24273162 0.21272242 0.5841096  0.38965252] 0.4436632
dumped
OI /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.41717756
COCO /m/083wq [0.7832434  0.74537426 0.98465556 0.9559081 ] 0.4171

In [32]:
det_results_combined_2 = {}
for key, value in det_results_combined.items():
    neg_probs = [-val['detection_score'] for val in value]
    sort_idx = np.argsort(neg_probs)
    det_results_combined_2[key] = {'detection_classes': [], 'detection_boxes': [], 'detection_scores': [], 'detection_models': []}
    for i in sort_idx:
        det_results_combined_2[key]['detection_classes'].append(value[i]['detection_class'])
        det_results_combined_2[key]['detection_boxes'].append(value[i]['detection_box'])
        det_results_combined_2[key]['detection_scores'].append(value[i]['detection_score'])
        det_results_combined_2[key]['detection_models'].append(value[i]['detection_model'])
    det_results_combined_2[key]['num_detections'] = len(sort_idx)
    
    

In [33]:
with open('../results/det_results_combined_1.pkl', 'wb') as fout:
    pickle.dump(det_results_combined_2, fout)